Qué hace el script:

*  Genera 1338 prestadores únicos con NITs simulados.
*  Aplica el Pareto (100 prestadores concentran 80% de facturas).
*  Distribuye las facturas por mes y semana según tus porcentajes.
*  Asigna valores dentro de los rangos indicados, cumpliendo los límites.
*  Ajusta los montos para que la facturación total sea ≈ $536.099.280.282.
*  Crea tanto la muestra (10.000 filas) como la opción para el dataset completo (659.748 filas).

El data set debe tener las columnas de:
•	ID prestador = el NIT del prestador
•	Nombre prestador
•	Numero de factura = alfanumérico
•	Fecha de radicación en formato DD/MM/AAAA
•	Valor factura = valores entre $5.000 y $800.000.000
•	Aplicativo en el cual se gestionó = aplicativo interno y aplicativo externo

Este codigo fue generado desde definiciones y variables propias, con la claridad de la necesida se generero con ChatGPT el codigo base, se comenzo a correr y se fue ajustando desde preguntas y pruebas.

Ajustes:
agregar cantidad de lienas de factura
agregar variable de tipo de factura, clasificación
incrementar los años
minimizar la cantidad de facturas

cantidad_lineas (entre 1 y 1200, con distribución controlada)

tipo_factura (Salud básica, Salud media, Salud compleja, Transporte)

~65.974 facturas por año (1/10 del total que definiste, para mantener proporción).

Facturación total promedio por año de $53.609.928.028.

Se mantienen los mismos prestadores, aplicativos, y lógica de tipo/cantidad de líneas.

Definición anual

Definimos año base (2021) con:

65_974 facturas

$53.609.928.028 facturación total

Cada año siguiente crece con pequeñas variaciones aleatorias dentro del rango que tú defines:

Facturas: +0% a +5%

Facturación: +0% a +3%

Se aplica un factor de crecimiento acumulado controlado.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta #datetime proporciona clases para manipular fechas y horas.
import random

In [ ]:
# ======================
# PARÁMETROS GENERALES
# ======================
TOTAL_FACTURAS = 65_974
TOTAL_FACTURAS_MUESTRA = 10_000
TOTAL_PRESTADORES = 1338
TOTAL_PRESTADORES_SOLO_INTERNO = 663
TOTAL_FACTURACION = 53_609_928_028

np.random.seed(42)


In [ ]:
# ======================
# GENERACIÓN DE PRESTADORES
# ======================
prestadores = pd.DataFrame({
    "id_prestador": np.arange(900000001, 900000001 + TOTAL_PRESTADORES),
    "nombre_prestador": [f"PRESTADOR_{i:04d}" for i in range(1, TOTAL_PRESTADORES + 1)]
})

In [ ]:
# Marcar prestadores solo internos y mixtos
prestadores["tipo"] = "Mixto"
prestadores.loc[:TOTAL_PRESTADORES_SOLO_INTERNO - 1, "tipo"] = "Interno"

In [ ]:
# Pareto: 100 prestadores concentran el 80% de las facturacón

prestadores["peso_facturacion"] = 0.2  #el 20% de la facturación
prestadores.loc[:99, "peso_facturacion"] = 0.8    #el 80% de la facturación

In [ ]:
prestadores.head()

,id_prestador,nombre_prestador,tipo,peso_facturacion
0,900000001,PRESTADOR_0001,Interno,0.8
1,900000002,PRESTADOR_0002,Interno,0.8
2,900000003,PRESTADOR_0003,Interno,0.8
3,900000004,PRESTADOR_0004,Interno,0.8
4,900000005,PRESTADOR_0005,Interno,0.8


In [ ]:
# ======================
# FUNCIÓN PARA FECHAS
# ======================
def generar_fecha_radicacion(anio):
    # Distribución semanal: 16%, 23%, 24%, 37%
    semanas = [0.16, 0.23, 0.24, 0.37]
    mes = random.randint(1, 12)
    dias_mes = (datetime(anio if mes < 12 else anio + 1, (mes % 12) + 1, 1) - timedelta(days=1)).day

    semana = np.random.choice([1, 2, 3, 4], p=semanas)
    if semana == 1:
        dia = random.randint(1, dias_mes // 4)
    elif semana == 2:
        dia = random.randint(dias_mes // 4 + 1, dias_mes // 2)
    elif semana == 3:
        dia = random.randint(dias_mes // 2 + 1, int(3 * dias_mes / 4))
    else:
        dia = random.randint(int(3 * dias_mes / 4) + 1, dias_mes)

    return datetime(anio, mes, min(dia, dias_mes)).strftime("%d/%m/%Y")


In [ ]:
# ======================
# FUNCIÓN PARA VALOR DE FACTURA
# ======================
def generar_valor_factura():
    # 65% hasta 5 millones
    if np.random.rand() < 0.65:
        return random.randint(10_000, 5_000_000)
    else:
        # 35% entre 5 y 800 millones
        valor = random.randint(5_000_001, 800_000_000)
        # Máximo 3 facturas por mes > 300 millones (se controla estadísticamente)
        return valor

In [ ]:
# FUNCIÓN TIPO DE FACTURA
# ======================
def generar_tipo_factura():
    tipos = ["Salud básica", "Salud media", "Salud compleja", "Transporte"]
    probabilidades = [0.45, 0.30, 0.15, 0.10]
    return np.random.choice(tipos, p=probabilidades)

In [ ]:
# FUNCIÓN CANTIDAD DE LÍNEAS
# ======================
def generar_cantidad_lineas(tipo_factura):
    p = np.random.rand()

    if tipo_factura == "Transporte":
        if p < 0.9:
            return random.randint(1, 5)
        else:
            return random.randint(6, 50)
    elif tipo_factura == "Salud básica":
        if p < 0.9:
            return random.randint(1, 200)
        elif p < 0.99:
            return random.randint(201, 800)
        else:
            return random.randint(801, 1200)
    elif tipo_factura == "Salud media":
        if p < 0.7:
            return random.randint(50, 300)
        elif p < 0.99:
            return random.randint(301, 800)
        else:
            return random.randint(801, 1200)
    else:  # Salud compleja
        if p < 0.6:
            return random.randint(100, 400)
        elif p < 0.99:
            return random.randint(401, 800)
        else:
            return random.randint(801, 1200)

In [ ]:
# ======================
# FUNCIÓN PARA GENERAR FACTURAS AÑO
# ======================
def generar_facturas_por_anio(anio, total_registros, total_facturacion):
    facturas = []
    for i in range(total_registros):
        prestador = prestadores.sample(weights=prestadores["peso_facturacion"], n=1).iloc[0]
        id_prestador = prestador["id_prestador"]
        nombre = prestador["nombre_prestador"]

        if prestador["tipo"] == "Interno":
            aplicativo = "Interno"
        else:
            aplicativo = np.random.choice(["Interno", "Externo"], p=[0.95, 0.05])

        tipo_factura = generar_tipo_factura()
        cantidad_lineas = generar_cantidad_lineas(tipo_factura)

        factura = {
            "anio": anio,
            "id_prestador": id_prestador,
            "nombre_prestador": nombre,
            "numero_factura": f"FAC-{anio}-{i+1:06d}",
            "fecha_radicacion": generar_fecha_radicacion(anio),
            "valor_factura": generar_valor_factura(),
            "aplicativo": aplicativo,
            "tipo_factura": tipo_factura,
            "cantidad_lineas": cantidad_lineas
        }
        facturas.append(factura)

    df = pd.DataFrame(facturas)
    factor = total_facturacion / df["valor_factura"].sum()
    df["valor_factura"] = (df["valor_factura"] * factor).round(0)
    return df


In [ ]:
anios = [2021, 2022, 2023, 2024, 2025]
df_total = pd.DataFrame()

facturas_anio = TOTAL_FACTURAS
facturacion_anio = TOTAL_FACTURACION

for anio in anios:
    print(f"Generando datos {anio}...")
    df_anio = generar_facturas_por_anio(anio, int(facturas_anio), facturacion_anio)
    df_total = pd.concat([df_total, df_anio], ignore_index=True)

    # Crecimiento aleatorio <5% en facturas, <3% en facturación
    crecimiento_facturas = np.random.uniform(0.0, 0.05)
    crecimiento_facturacion = np.random.uniform(0.0, 0.03)
    facturas_anio *= (1 + crecimiento_facturas)
    facturacion_anio *= (1 + crecimiento_facturacion)

Generando datos 2021...
Generando datos 2022...
Generando datos 2023...
Generando datos 2024...
Generando datos 2025...


In [ ]:
# GUARDAR RESULTADO
# ======================
df_total.to_csv("dataset_facturacion_2021_2025_crecimiento.csv", index=False, encoding="utf-8-sig")
print("Archivo generado: dataset_facturacion_2021_2025_crecimiento.csv")

Archivo generado: dataset_facturacion_2021_2025_crecimiento.csv


In [ ]:
#ejecutar
#!head es para mostrar las primeras lineas.

!head dataset_facturacion_2021_2025_crecimiento.csv

﻿anio,id_prestador,nombre_prestador,numero_factura,fecha_radicacion,valor_factura,aplicativo,tipo_factura,cantidad_lineas
2021,900000314,PRESTADOR_0314,FAC-2021-000001,19/05/2021,4621.0,Interno,Transporte,1
2021,900000064,PRESTADOR_0064,FAC-2021-000002,20/08/2021,2485269.0,Interno,Salud básica,110
2021,900000009,PRESTADOR_0009,FAC-2021-000003,10/12/2021,21441.0,Interno,Transporte,5
2021,900000076,PRESTADOR_0076,FAC-2021-000004,17/11/2021,9051.0,Interno,Salud básica,66
2021,900000703,PRESTADOR_0703,FAC-2021-000005,18/05/2021,4240179.0,Interno,Salud básica,157
2021,900000082,PRESTADOR_0082,FAC-2021-000006,04/06/2021,18999.0,Interno,Salud media,90
2021,900000070,PRESTADOR_0070,FAC-2021-000007,25/01/2021,2976606.0,Interno,Salud básica,307
2021,900000199,PRESTADOR_0199,FAC-2021-000008,22/03/2021,18635.0,Interno,Salud básica,23
2021,900000512,PRESTADOR_0512,FAC-2021-000009,08/06/2021,179011.0,Interno,Salud básica,746


In [ ]:
#Archivo de la muestra
from google.colab import files
files.download("dataset_facturacion_2021_2025_crecimiento.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  Link

Link: https://bootcamp-ia.onrender.com/

